In [1]:
import os
import sys
import pickle 
import dgl
import dill
import pickle
import pandas as pd
import numpy as np
from imp import reload
import math
import h5py
import shutil
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from itertools import chain
from sklearn.model_selection import KFold
from progiter import ProgIter
from mymodule import Trasnformer_model, VAE_model, GAE_model
import seaborn as sns
from scipy.linalg import svd



import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.nn.functional import binary_cross_entropy_with_logits as BCELoss

random_state=34

Using backend: pytorch


In [2]:
with open('/tf/notebooks/code_for_pub/input_files/mols_drugcombonly.pkl', 'rb') as f:
    mols = pickle.load(f)
with open("/tf/notebooks/code_for_pub/input_files/graphs_drugcombonly.pkl", "rb") as f:
    graphs = pickle.load(f)

In [3]:
PATH = '/tf/notebooks/code_for_pub/_logs_as_python_files/gae_training_logs/keep_ep189'
device = torch.device('cpu')
m = torch.load(PATH, map_location=device)

In [4]:
#model = GAE_model.GAE(in_dim=54, hidden_dims=[54,45,35,31,27,23,20,16]) # previous best
PATH = '/tf/notebooks/code_for_pub/_logs_as_python_files/gae_training_logs/ep200'
device = torch.device('cpu')


m = torch.load(PATH, map_location=device)
model = GAE_model.GAE(in_dim=54, hidden_dims=[54,46,40,34,28,22,16]
)
model.load_state_dict(m)
model.eval()

GAE(
  (layers): ModuleList(
    (0): GCN(
      (linear): Linear(in_features=54, out_features=54, bias=True)
    )
    (1): GCN(
      (linear): Linear(in_features=54, out_features=46, bias=True)
    )
    (2): GCN(
      (linear): Linear(in_features=46, out_features=40, bias=True)
    )
    (3): GCN(
      (linear): Linear(in_features=40, out_features=34, bias=True)
    )
    (4): GCN(
      (linear): Linear(in_features=34, out_features=28, bias=True)
    )
    (5): GCN(
      (linear): Linear(in_features=28, out_features=22, bias=True)
    )
    (6): GCN(
      (linear): Linear(in_features=22, out_features=16, bias=True)
    )
  )
  (decoder): InnerProductDecoder()
)

In [6]:
d_GAE=[]

for i in graphs:
    a = model.encode(i).detach().numpy()
    d_GAE.append(a)

In [7]:

# this is the length of the GAE fp, since we do min agg, max agg, mean agg, and SVD decomposition
fps_GAE = np.zeros((len(d_GAE),16*4), dtype=np.float) 

for index,i in enumerate(d_GAE):
    me = np.mean(i, axis=0) # aggregation over representation
    mi = np.min(i, axis=0)
    ma = np.max(i, axis=0)
    sv = svd(i, compute_uv=False)
    if sv.shape[0] != 16: 
        # zero padding, in case of small molecules
        result = np.zeros((16,))
        result[:sv.shape[0]] = sv
        sv = result.copy()
    fp = np.append([me,mi,ma],sv)
    fps_GAE[index] = fp
fps_GAE = pd.DataFrame(fps_GAE)

In [13]:
# use different fps to copy index
with open('/tf/notebooks/code_for_pub/fp_files/fps_VAE_16bit_new.pickle', 'rb') as f:
    drugs = pickle.load(f)
fps_GAE.index = drugs.index

In [15]:
with open('/tf/notebooks/code_for_pub/fp_files/fps_gae_64bit_new.pickle', 'wb') as f:
    pickle.dump(fps_GAE,f)